In [ ]:
#crop the pdf

In [22]:
# !pip install tabula-py
# !pip install MySQL-python
# !pip install mysqlclient
# !pip install pandas sqlalchemy mysql-connector-python

You should consider upgrading via the 'C:\Users\0045H7744\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
import pandas as pd
import numpy as np
from pandas.io import sql
import MySQLdb
from sqlalchemy import create_engine

In [2]:
DB_USERNAME = 'root'
DB_PASSWORD = '12345678'
DB_HOST = '127.0.0.1'
DB_PORT = '3306'
DB_NAME = 'expense_manager'

# Create the SQLAlchemy engine
engine = create_engine(f'mysql+mysqlconnector://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')


In [3]:
# file_path = "C:/Users/0045h7744/Downloads/Statements (2)-1.pdf"
file_path = "C:/Users/0045h7744/Downloads/Statements (3)-1.pdf"
# file_path = "C:/Users/0045h7744/Downloads/Statements-1.pdf"

statement_year = '/2023'
customer_id = '2'

In [4]:
categories = {
    "Rent": ["rent", "lease"],
    "Grocery": ["supermarket", "groceries", "grocery", "market","Walmart.com","Grocers"],
    "Food": ["restaurant", "dinner", "lunch", "breakfast", "Mcdonalds"],
    "Entertainment": ["movie", "tickets", "cinema", "theater"],
    "Bills": ["bill", "utility", "electricity", "water","petrol","fuel","Wsu"],
    "Medical": ["doctor", "visit", "hospital", "health"],
    "Miscellaneous": ["miscellaneous", "others", "misc"],
    "Investment": ["Insurance"],
}


In [5]:
os.path.exists(file_path)

True

In [6]:
#change the pages according to the pdf
from tabula import read_pdf
from tabulate import tabulate

#reads table from pdf file
df = read_pdf(file_path,pages=2,output_format="dataframe") #address of pdf file
# tabulate(df)


In [7]:
df[0].reset_index().T.reset_index().T.reset_index(drop=True)

,0,1,2,3,4,5
0,index,10/30,Card Purchase 10/28 Parcel Pending 855-977-167...,Unnamed: 2,-20.00,155.87
1,0,10/31,Zelle Payment To Santosh Wsu 18875061991,NaN,-10.42,145.45
2,1,11/01,Zelle Payment From Siddhartha Nadimpalli Baca6...,NaN,400.0,545.45
3,2,11/02,Zelle Payment To Siddhartha Jpm99A4Fh8Wo,NaN,-120.0,425.45
4,3,11/06,Zelle Payment From Mourya Gottipati 18923567502,NaN,169.0,594.45
5,4,11/06,Zelle Payment From Sai Charan Kommuri 18921852620,NaN,155.87,750.32
6,5,11/06,Zelle Payment From Moses Raj Muddada 18927411171,NaN,70.0,820.32
7,6,11/06,Card Purchase 11/05 Ysi*Sl Province Owner 732-...,NaN,-627.99,192.33
8,7,NaN,9061,NaN,NaN,NaN
9,8,11/13,Card Purchase 11/12 Temu.Com 130-248-0611 MA C...,NaN,-13.46,178.87


In [8]:
#based on the above drop the additional columns-->0 is default index to be dropped
transactions = df[0].reset_index().T.reset_index().T.reset_index(drop=True).drop([0,3],axis=1)

In [9]:
transactions

,1,2,4,5
0,10/30,Card Purchase 10/28 Parcel Pending 855-977-167...,-20.00,155.87
1,10/31,Zelle Payment To Santosh Wsu 18875061991,-10.42,145.45
2,11/01,Zelle Payment From Siddhartha Nadimpalli Baca6...,400.0,545.45
3,11/02,Zelle Payment To Siddhartha Jpm99A4Fh8Wo,-120.0,425.45
4,11/06,Zelle Payment From Mourya Gottipati 18923567502,169.0,594.45
5,11/06,Zelle Payment From Sai Charan Kommuri 18921852620,155.87,750.32
6,11/06,Zelle Payment From Moses Raj Muddada 18927411171,70.0,820.32
7,11/06,Card Purchase 11/05 Ysi*Sl Province Owner 732-...,-627.99,192.33
8,NaN,9061,NaN,NaN
9,11/13,Card Purchase 11/12 Temu.Com 130-248-0611 MA C...,-13.46,178.87


In [10]:
def concat_empty_rows(df,date_col,description_col):
    result = []
    temp_str = ''
    for index, row in df.iterrows():
        if pd.isnull(row[date_col]):
            temp_str += str(row[description_col])
        else:
            if temp_str:
                result[-1][description_col] += str(temp_str)
                temp_str = ''
            result.append(row)
    if temp_str:
        result[-1][description_col] += temp_str
    return pd.DataFrame(result).reset_index(drop=True)

In [11]:

# Function to calculate similarity score
def calculate_similarity(description, keywords):
    # Simple string matching for any substring
    score = sum(keyword.lower() in description.lower() for keyword in keywords)
    return score

def get_category(transactions, description_col, categories):
    # Classify transactions
    category_col = []
    for description in transactions[description_col]:
        max_score = 0
        assigned_category = "Uncategorized"
        for category, keywords in categories.items():
            similarity_score = calculate_similarity(description, keywords)
            if similarity_score > max_score:
                max_score = similarity_score
                assigned_category = category
#         print(f"Transaction: {transaction}  Category: {assigned_category}")
        category_col.append(assigned_category)
    return category_col


In [12]:
def process_transaction(transactions, rows_to_delete, categories, statement_year, customer_id):
    transactions = transactions.iloc[rows_to_delete:]
    transactions.columns = ['transaction_date','transaction_description','amount','balance']
    transactions = concat_empty_rows(transactions,'transaction_date','transaction_description')
    transactions['category'] = get_category(transactions, 'transaction_description', categories)
    transactions['transaction_date'] = transactions['transaction_date'].astype(str) + statement_year
    transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
    if transactions['amount'].dtype == 'object':
        transactions['amount'] = transactions['amount'].astype(str).str.replace(',','').astype(float)
    transactions = transactions.drop(['balance'],axis=1)
    transactions['tag'] = np.where(transactions['amount']<0,'Expense','Income')
    transactions['amount'] = abs(transactions['amount'])
    transactions['customer_id'] = customer_id
    return transactions

In [13]:
transactions = process_transaction(transactions,0,categories,statement_year,customer_id)

In [ ]:
table_name = 'transactions'

try:
    # Append DataFrame to the existing table in MySQL
    transactions.to_sql(name=table_name, con=engine, if_exists='append', index=False)
    print("Data appended successfully.")
except Exception as e:
    print("An error occurred:", e)

In [16]:
pd.read_sql_table(table_name, engine)

,transaction_id,transaction_date,transaction_description,amount,category,tag,customer_id
0,1,2023-11-27,Card Purchase Return 11/24 Lemonade Insurance ...,9.19,Investment,Income,1
1,2,2023-11-27,Card Purchase 11/26 Walmart.Com 800-966-6546 A...,11.32,Grocery,Expense,1
2,3,2023-12-04,Zelle Payment From Siddhartha Nadimpalli Bacca...,300.00,Uncategorized,Income,1
3,4,2023-12-04,Zelle Payment From Mourya Gottipati 19197247021,282.31,Uncategorized,Income,1
4,5,2023-12-04,Card Purchase 12/02 Samsclub.Com 888-746-7726 ...,39.36,Uncategorized,Expense,1
5,6,2023-12-05,Zelle Payment From Moses Raj Muddada 19208663192,283.00,Uncategorized,Income,1
6,7,2023-12-05,Zelle Payment From Ali Mohammed Pncaa0Iqf79P,10.00,Uncategorized,Income,1
7,8,2023-12-06,Zelle Payment From Ali Mohammed Pncaa0Iqe81Y,2500.00,Uncategorized,Income,1
8,9,2023-12-06,Zelle Payment From Vishnuvardhan Reddy Sama 19...,96.00,Uncategorized,Income,1
9,10,2023-12-06,Zelle Payment From Sai Charan Kommuri 19209061760,24.00,Uncategorized,Income,1


### Trash

In [3]:
# con = MySQLdb.connect(host="127.0.0.1",user="root",passwd="12345678",port=3306,database="expense_manager") 

In [6]:
# import camelot

# # extract all the tables in the PDF file
# abc = camelot.read_pdf(file_path) #address of file location

# # print the first table as Pandas DataFrame
# print(abc[0].df)


In [129]:
#based on the above delete the initial rows
transactions = transactions.iloc[2:]

In [132]:
transactions.columns = ['transaction_date','transaction_description','amount','balance']

In [134]:
transactions = concat_empty_rows(transactions,'transaction_date','transaction_description')

In [136]:
transactions['category'] = get_category(transactions, 'transaction_description', categories)

In [137]:
transactions['transaction_date'] = transactions['transaction_date'].astype(str) + statement_year

In [138]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])

In [139]:
transactions

,transaction_date,transaction_description,amount,balance,category
0,2024-10-30,Card Purchase 10/28 Parcel Pending 855-977-167...,-20.00,155.87,Uncategorized
1,2024-10-31,Zelle Payment To Santosh Wsu 18875061991,-10.42,145.45,Bills
2,2024-11-01,Zelle Payment From Siddhartha Nadimpalli Baca6...,400.0,545.45,Uncategorized
3,2024-11-02,Zelle Payment To Siddhartha Jpm99A4Fh8Wo,-120.0,425.45,Uncategorized
4,2024-11-06,Zelle Payment From Mourya Gottipati 18923567502,169.0,594.45,Uncategorized
5,2024-11-06,Zelle Payment From Sai Charan Kommuri 18921852620,155.87,750.32,Uncategorized
6,2024-11-06,Zelle Payment From Moses Raj Muddada 18927411171,70.0,820.32,Uncategorized
7,2024-11-06,Card Purchase 11/05 Ysi*Sl Province Owner 732-...,-627.99,192.33,Uncategorized
8,2024-11-13,Card Purchase 11/12 Temu.Com 130-248-0611 MA C...,-13.46,178.87,Uncategorized
9,2024-11-13,Recurring Card Purchase 11/13 Lemonade Insuran...,-15.75,163.12,Investment


In [140]:
transactions['amount'].dtypes == 'object'

True

In [141]:
if transactions['amount'].dtype == 'object':
#     try:
        transactions['amount'] = transactions['amount'].astype(str).str.replace(',','').astype(float)
#     except:
#         transactions['amount'] = transactions['amount'].astype(float)
# if transactions['Balance'].dtype == 'object':
#     try:
#         transactions['Balance'] = transactions['Balance'].str.replace(',','').astype(float)
#     except AttributeError as e:
#         transactions['Balance'] = transactions['Balance'].astype(float)


In [145]:
transactions.drop(['balance'],axis=1,inplace=True)

In [146]:
transactions['tag'] = np.where(transactions['amount']<0,'Expense','Income')
transactions['amount'] = abs(transactions['amount'])

In [148]:
transactions['customer_id'] = customer_id

In [169]:
transactions

,transaction_date,transaction_description,amount,category,tag,customer_id
0,2023-10-30,Card Purchase 10/28 Parcel Pending 855-977-167...,20.00,Uncategorized,Expense,2
1,2023-10-31,Zelle Payment To Santosh Wsu 18875061991,10.42,Bills,Expense,2
2,2023-11-01,Zelle Payment From Siddhartha Nadimpalli Baca6...,400.00,Uncategorized,Income,2
3,2023-11-02,Zelle Payment To Siddhartha Jpm99A4Fh8Wo,120.00,Uncategorized,Expense,2
4,2023-11-06,Zelle Payment From Mourya Gottipati 18923567502,169.00,Uncategorized,Income,2
5,2023-11-06,Zelle Payment From Sai Charan Kommuri 18921852620,155.87,Uncategorized,Income,2
6,2023-11-06,Zelle Payment From Moses Raj Muddada 18927411171,70.00,Uncategorized,Income,2
7,2023-11-06,Card Purchase 11/05 Ysi*Sl Province Owner 732-...,627.99,Uncategorized,Expense,2
8,2023-11-13,Card Purchase 11/12 Temu.Com 130-248-0611 MA C...,13.46,Uncategorized,Expense,2
9,2023-11-13,Recurring Card Purchase 11/13 Lemonade Insuran...,15.75,Investment,Expense,2


In [170]:
table_name = 'transactions'

try:
    # Append DataFrame to the existing table in MySQL
    transactions.to_sql(name=table_name, con=engine, if_exists='append', index=False)
    print("Data appended successfully.")
except Exception as e:
    print("An error occurred:", e)

# try:
#     # Insert DataFrame into MySQL table
#     transactions.to_sql(name=table_name, con=engine, if_exists='append', index=False)
#     print("Data inserted successfully.")
# except Exception as e:
#     print(f"An error occurred: {e}")
#     print("Skipping rows with problematic data...")
#     for idx, row in df.iterrows():
#         try:
#             row.to_frame().transpose().to_sql(name=table_name, con=engine, if_exists='append', index=False)
#         except Error as e:
#             print(f"Error inserting row {idx}: {e}")
# print("Insertion complete.")


Data appended successfully.


In [162]:
pd.read_sql_table(table_name, engine)

,transaction_id,transaction_date,transaction_description,amount,category,tag,customer_id
0,1,2023-11-27,Card Purchase Return 11/24 Lemonade Insurance ...,9.19,Investment,Income,1
1,2,2023-11-27,Card Purchase 11/26 Walmart.Com 800-966-6546 A...,11.32,Grocery,Expense,1
2,3,2023-12-04,Zelle Payment From Siddhartha Nadimpalli Bacca...,300.00,Uncategorized,Income,1
3,4,2023-12-04,Zelle Payment From Mourya Gottipati 19197247021,282.31,Uncategorized,Income,1
4,5,2023-12-04,Card Purchase 12/02 Samsclub.Com 888-746-7726 ...,39.36,Uncategorized,Expense,1
5,6,2023-12-05,Zelle Payment From Moses Raj Muddada 19208663192,283.00,Uncategorized,Income,1
6,7,2023-12-05,Zelle Payment From Ali Mohammed Pncaa0Iqf79P,10.00,Uncategorized,Income,1
7,8,2023-12-06,Zelle Payment From Ali Mohammed Pncaa0Iqe81Y,2500.00,Uncategorized,Income,1
8,9,2023-12-06,Zelle Payment From Vishnuvardhan Reddy Sama 19...,96.00,Uncategorized,Income,1
9,10,2023-12-06,Zelle Payment From Sai Charan Kommuri 19209061760,24.00,Uncategorized,Income,1


In [50]:
df = pd.read_sql_table(table_name, engine)

In [51]:
df.groupby('category')['amount'].sum().reset_index()

,category,amount
0,Bills,6467.80
1,Food,21.08
2,Grocery,65.21
3,Investment,9.19
4,Uncategorized,8443.44


In [64]:
# transactions.to_csv('test_transaction.csv',index=False)